# Examples
In this notebook we will see an example for each prompting strategy of how an explanation is generated and evaluated from start to finish

In [1]:
import dice_ml
from dice_ml.utils import helpers
import pandas as pd
import pickle
from prompts import *
from utils import *
from llm_explainers import *

In [2]:
# Load the model we want to explain
with open("""./models/loan_model.pkl""", 'rb') as file:
    model = pickle.load(file)

#Load the train and test data set
train_dataset = pd.read_csv('./data/adult_train_dataset.csv')
test_dataset = pd.read_csv('./data/adult_test_dataset.csv')

#Load the examples we will try to explain
test_df = pd.read_csv('./data/test_examples.csv')
test_df = test_df.drop(columns=['income'], axis = 1)

## Zero Shot

In [5]:
exp_m = LLMExplanation4CFs(model = model, #Load the model we want to explain
                            model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                            backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                            dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                            continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                            outcome_name= 'income', #Necessary for counterfactual generation
                            training_set=train_dataset, #Necessary for counterfactual generation
                            test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                            llm='gpt-4o', #LLM used, works with Langchain
                            prompt_type='zero', # zero or one
                            n_counterfactuals=5, #Number of counterfactuals used in the explanation 
                            user_input=False #Human in the loop helping select the causes
                           )


exp_m.fit()
counterfactuals, rules, code1, result1, explanation, code2, final_cf, code3, prediction, n_rules,rules_followed, first_rule, second_rule,third_rule,  is_in_data = exp_m.explain_evaluate(user_data = test_df.iloc[[0]], verbose = False,return_all=True)

100%|██████████| 1/1 [00:00<00:00,  1.80it/s]


We will looked at the following example of a woman who is predicted to earn less than 50k$ a year. We will look at the whole process followed by the LLM in order to obtain this final explanation.

In [6]:
print(test_df.iloc[0].to_string)

<bound method Series.to_string of age                        29
workclass             Private
education             HS-grad
marital_status        Married
occupation        Blue-Collar
race                    White
gender                 Female
hours_per_week             38
Name: 0, dtype: object>


First, a set of counterfactuals will be generated using the DiCE ML package

In [7]:
counterfactuals

<bound method DataFrame.to_string of    age      workclass    education marital_status     occupation   race  \
0   29        Private    Bachelors        Married  Other/Unknown  White   
1   29  Other/Unknown      HS-grad        Married   Professional  White   
2   29        Private  Prof-school        Married    Blue-Collar  White   
3   29     Government      HS-grad        Married   Professional  White   
4   29  Self-Employed      HS-grad        Married   White-Collar  White   

   gender  hours_per_week  income  
0  Female              38       1  
1  Female              38       1  
2    Male              38       1  
3  Female              38       1  
4  Female              38       1  >

A set of rules is extracted from this counterfactual using the LLM

In [8]:
print(rules)

Based on the provided negative assessment outcome and the positive counterfactual outcomes, we can derive the following important rules that potentially lead to a positive income prediction for a 29-year-old individual:

### Observed Rules to Achieve a Positive Income Prediction:

1. **Education Level**:
    - Upgrading education from "HS-grad" to "Bachelors" or higher (e.g., "Prof-school") can lead to a positive income prediction.
    - Example from counterfactuals:
        - Case 0: Education changed from "HS-grad" to "Bachelors".
        - Case 2: Education changed from "HS-grad" to "Prof-school".

2. **Occupation**:
    - Changing occupation from "Blue-Collar" to other types of occupations, such as "Other/Unknown," "Professional," or "White-Collar," can lead to a positive income prediction.
    - Example from counterfactuals:
        - Case 0: Occupation changed from "Blue-Collar" to "Other/Unknown".
        - Case 1: Occupation changed from "Blue-Collar" to "Professional".
       

In order to check whether this rules are correct or not, we ask the LLM to create a program that checks it. In the following cell we can see the code generated by the LLM.

In [9]:
print(code1)

import pandas as pd

# Negative assessment outcome
negative_outcome = pd.DataFrame({
    'age': [29],
    'workclass': ['Private'],
    'education': ['HS-grad'],
    'marital_status': ['Married'],
    'occupation': ['Blue-Collar'],
    'race': ['White'],
    'gender': ['Female'],
    'hours_per_week': [38],
    'income': [0]
})

# Positive counterfactual outcomes
positive_counterfactuals = pd.DataFrame({
    'age': [29, 29, 29, 29, 29],
    'workclass': ['Private', 'Other/Unknown', 'Private', 'Government', 'Self-Employed'],
    'education': ['Bachelors', 'HS-grad', 'Prof-school', 'HS-grad', 'HS-grad'],
    'marital_status': ['Married', 'Married', 'Married', 'Married', 'Married'],
    'occupation': ['Other/Unknown', 'Professional', 'Blue-Collar', 'Professional', 'White-Collar'],
    'race': ['White', 'White', 'White', 'White', 'White'],
    'gender': ['Female', 'Female', 'Male', 'Female', 'Female'],
    'hours_per_week': [38, 38, 38, 38, 38],
    'income': [1, 1, 1, 1, 1]
})

# Rules
ru

After executing this code, the following results were obtained

In [10]:
print(result1)

Number of counterfactuals consistent with each rule:
Occupation: 4
Workclass: 3
Education Level: 2
Gender: 1



By using this causes, the LLM will produce a final explanation, hopefully using the most important causes

In [11]:
print(explanation)

### Explanation:

Based on the analysis of the data and the rules derived from counterfactual outcomes, here are some actionable steps you can take to improve your chances of earning more than $50k a year. The steps are prioritized based on how frequently they were observed to be effective in the counterfactual cases.

1. **Change Your Occupation**:
   - **Most Important**: The data suggests that changing your occupation from "Blue-Collar" to another type such as "Professional," "White-Collar," or even "Other/Unknown" can significantly increase your chances of earning more than $50k a year. 
   - **Action**: Consider seeking opportunities for training or education that can qualify you for these types of occupations. Networking and exploring job openings in these areas may also be beneficial.

2. **Consider Different Work Environments**:
   - **Highly Important**: Switching your workclass from "Private" to "Government," "Self-Employed," or even "Other/Unknown" has been shown to positive

Now we would like to check the quality of our explanation. As we explain in the paper, we created a close loop evaluation method that checks whether the 

In [12]:
print(code2)

import pandas as pd

# Creating a DataFrame with values that are likely to fall into the positive class
df = pd.DataFrame({
    'age': [29],
    'workclass': ['Government'],  # Changed from 'Private' to 'Government'
    'education': ['Bachelors'],  # Changed from 'HS-grad' to 'Bachelors'
    'marital_status': ['Married'],
    'occupation': ['Professional'],  # Changed from 'Blue-Collar' to 'Professional'
    'race': ['White'],
    'gender': ['Female'],
    'hours_per_week': [38],
    'income': [1]  # Positive class (>50K)
})

# Saving the DataFrame to a CSV file
df.to_csv('temp_csv.csv', index=False)


After executing the code the LLM correctly generated a counterfactual example.

In [13]:
final_cf

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
0,29,Government,Bachelors,Married,Professional,White,Female,38


We will now try to extract metrics from this counterfactual like the predicted class, rules followed and if it exists in the data set. We ask the LLM for code again and this time it will create a table that we can analyze.

In [14]:
print(code3)

import pandas as pd

# Load the counterfactual example from temp_csv.csv
df = pd.read_csv('temp_csv.csv')

# Define the rules and their importance
rules = [
    {"Rule": "Increase Education Level from 'HS-grad' to 'Bachelors' or 'Prof-school'", "Importance": 2, "Column": "education", "Values": ["Bachelors", "Prof-school"]},
    {"Rule": "Change Occupation from 'Blue-Collar' to 'Other/Unknown', 'Professional', or 'White-Collar'", "Importance": 4, "Column": "occupation", "Values": ["Other/Unknown", "Professional", "White-Collar"]},
    {"Rule": "Change Workclass from 'Private' to 'Other/Unknown', 'Government', or 'Self-Employed'", "Importance": 3, "Column": "workclass", "Values": ["Other/Unknown", "Government", "Self-Employed"]},
    {"Rule": "Change Gender from 'Female' to 'Male'", "Importance": 1, "Column": "gender", "Values": ["Male"]}
]

# Initialize the final dataframe
results = []

# Check if the counterfactual example follows each rule
for rule in rules:
    rule_followed = int(df

In [15]:
eval_df = pd.read_csv('./temp_files/evaluation.csv')
eval_df

,Rule,Importance,In explanation
0,Increase Education Level from 'HS-grad' to 'Ba...,2,1
1,Change Occupation from 'Blue-Collar' to 'Other...,4,1
2,Change Workclass from 'Private' to 'Other/Unkn...,3,1
3,Change Gender from 'Female' to 'Male',1,0


After analyzing the table we get the following results.

In [16]:
print('Prediction of the generated example: ', prediction)
print('Number of rules generated: ', n_rules)
print('Number of rules followed: ', rules_followed)
print('1st rule followed ', first_rule)
print('2nd rule followed ', second_rule)
print('3rd rule followed ', third_rule)
print('Example exists in data: ', is_in_data)

Prediction of the generated example:  1
Number of rules generated:  4
Number of rules followed:  3
1st rule followed  1
2nd rule followed  1
3rd rule followed  1
Example exists in data:  False


## One Shot

In [95]:
exp_m = LLMExplanation4CFs(model = model, #Load the model we want to explain
                            model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                            backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                            dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                            continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                            outcome_name= 'income', #Necessary for counterfactual generation
                            training_set=train_dataset, #Necessary for counterfactual generation
                            test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                            llm='gpt-4o', #LLM used, works with Langchain
                            prompt_type='one', # zero or one
                            n_counterfactuals=5, #Number of counterfactuals used in the explanation 
                            user_input=False #Human in the loop helping select the causes
                           )


exp_m.fit()
counterfactuals, rules, code1, result1, explanation, code2, final_cf, code3, prediction, n_rules,rules_followed, first_rule, second_rule,third_rule, is_in_data = exp_m.explain_evaluate(user_data = test_df.iloc[[2]], verbose = False,return_all=True)

100%|██████████| 1/1 [00:00<00:00,  4.78it/s]


We will looked at the following example of a woman who is predicted to earn less than 50k$ a year. We will look at the whole process followed by the LLM in order to obtain this final explanation.

In [96]:
print(test_df.iloc[2])

age                        41
workclass             Private
education              School
marital_status        Married
occupation        Blue-Collar
race                    White
gender                   Male
hours_per_week             30
Name: 2, dtype: object


First, a set of counterfactuals will be generated using the DiCE ML package

In [97]:
counterfactuals

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,41,Private,Prof-school,Married,Blue-Collar,White,Male,16,1
1,41,Private,School,Divorced,Professional,White,Male,30,1
2,41,Private,Bachelors,Married,Blue-Collar,White,Male,19,1
3,41,Self-Employed,School,Married,Blue-Collar,White,Female,30,1
4,41,Private,Doctorate,Married,Blue-Collar,White,Male,30,1


A set of rules is extracted from this counterfactual using the LLM

In [98]:
print(rules)

Based on the provided negative assessment outcome and the positive counterfactual outcomes, here are the most important observed rules:

1. **Higher Education (Prof-school, Bachelors, Doctorate) Leads to Higher Income**:
    - The individual in the negative outcome has a "School" level of education.
    - In the positive counterfactuals, individuals with "Prof-school", "Bachelors", and "Doctorate" education levels are shown to have a higher income.

2. **Marital Status Being Divorced Can Lead to Higher Income**:
    - In the counterfactual case where the individual is "Divorced", the income is higher even with the same "School" level of education and same occupation.

3. **Type of Workclass Can Influence Income**:
    - The individual in the negative outcome is in the "Private" workclass.
    - One of the counterfactuals shows that being "Self-Employed" with the same education level leads to a higher income.

4. **Occupation Being Professional Leads to Higher Income**:
    - The indivi

In order to check whether this rules are correct or not, we ask the LLM to create a program that checks it. In the following cell we can see the code generated by the LLM.

In [99]:
print(code1)

import pandas as pd

# Original negative outcome
original = pd.DataFrame({
    'age': [41],
    'workclass': ['Private'],
    'education': ['School'],
    'marital_status': ['Married'],
    'occupation': ['Blue-Collar'],
    'race': ['White'],
    'gender': ['Male'],
    'hours_per_week': [30],
    'income': [0]
})

# Counterfactuals
counterfactuals = pd.DataFrame({
    'age': [41, 41, 41, 41, 41],
    'workclass': ['Private', 'Private', 'Private', 'Self-Employed', 'Private'],
    'education': ['Prof-school', 'School', 'Bachelors', 'School', 'Doctorate'],
    'marital_status': ['Married', 'Divorced', 'Married', 'Married', 'Married'],
    'occupation': ['Blue-Collar', 'Professional', 'Blue-Collar', 'Blue-Collar', 'Blue-Collar'],
    'race': ['White', 'White', 'White', 'White', 'White'],
    'gender': ['Male', 'Male', 'Male', 'Female', 'Male'],
    'hours_per_week': [16, 30, 19, 30, 30],
    'income': [1, 1, 1, 1, 1]
})

# Rule 1: Higher education (Prof-school, Bachelors, Doctorate) lead

After executing this code, the following results were obtained

In [100]:
print(result1)

Number of counterfactuals following Rule 1 (Higher education): 3
Number of counterfactuals following Rule 2 (Marital status being Divorced): 1
Number of counterfactuals following Rule 3 (Self-Employed workclass): 1
Number of counterfactuals following Rule 4 (Occupation being Professional): 1



By using this causes, the LLM will produce a final explanation, hopefully using the most important causes

In [101]:
print(explanation)

Based on the analysis of your current situation and the observed patterns in the data, here are some clear steps you can take to improve your chances of earning more than $50K a year:

### 1. Pursue Higher Education
The most influential factor is education. Individuals with higher education levels such as Professional School, Bachelors, and Doctorate degrees are more likely to earn a higher income. Currently, you have a "School" level of education. Investing in further education could significantly improve your economic prospects. Consider enrolling in higher education programs or professional courses to enhance your qualifications.

### 2. Explore Different Occupations
Another key factor is the type of occupation. Individuals in "Professional" roles tend to earn higher incomes compared to those in "Blue-Collar" jobs. If possible, seek opportunities or training that could transition you into a more professional role within your field or a new industry that values your skills.

### 3. C

Now we would like to check the quality of our explanation. As we explain in the paper, we created a close loop evaluation method that checks whether the 

In [102]:
print(code2)

import pandas as pd

# Define the data for the DataFrame
data = {
    'age': [41],
    'workclass': ['Self-Employed'],
    'education': ['Bachelors'],
    'marital_status': ['Married'],
    'occupation': ['Professional'],
    'race': ['White'],
    'gender': ['Male'],
    'hours_per_week': [30],
    'income': [1]
}

# Create the DataFrame
df = pd.DataFrame(data)

# Save to csv
df.to_csv('temp_csv.csv', index=False)


After executing the code the LLM correctly generated a counterfactual example.

In [103]:
final_cf

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
0,41,Self-Employed,Bachelors,Married,Professional,White,Male,30


We will now try to extract metrics from this counterfactual like the predicted class, rules followed and if it exists in the data set. We ask the LLM for code again and this time it will create a table that we can analyze.

In [104]:
print(code3)

import pandas as pd

# Read example
df = pd.read_csv('temp_csv.csv')

# Define the data for the DataFrame
data = {
    'Rule': [
        'Higher education (Prof-school, Bachelors, Doctorate) leads to higher income.',
        'Marital status being Divorced can lead to higher income.',
        'Type of workclass being Self-Employed can influence income.',
        'Occupation being Professional leads to higher income.',
        'Hours per week less influential in this case.'
    ],
    'Importance': [3, 1, 1, 1, 0],  # Counterfactuals following each rule
    'In explanation': [0, 0, 0, 0, 0]  # Initial values, will update based on the example
}

# Create the DataFrame
df_final = pd.DataFrame(data)

# Check if the example follows each rule
if df['education'].iloc[0] in ['Prof-school', 'Bachelors', 'Doctorate']:
    df_final.at[0, 'In explanation'] = 1
if df['marital_status'].iloc[0] == 'Divorced':
    df_final.at[1, 'In explanation'] = 1
if df['workclass'].iloc[0] == 'Self-Employed':
    d

In [105]:
eval_df = pd.read_csv('./temp_files/evaluation.csv')
eval_df

,Rule,Importance,In explanation
0,"Higher education (Prof-school, Bachelors, Doct...",3,1
1,Marital status being Divorced can lead to high...,1,0
2,Type of workclass being Self-Employed can infl...,1,1
3,Occupation being Professional leads to higher ...,1,1
4,Hours per week less influential in this case.,0,1


After analyzing the table we get the following results.

In [106]:
print('Prediction of the generated example: ', prediction)
print('Number of rules generated: ', n_rules)
print('Number of rules followed: ', rules_followed)
print('1st rule followed ', first_rule)
print('2nd rule followed ', second_rule)
print('3rd rule followed ', third_rule)
print('Example exists in data: ', is_in_data)

Prediction of the generated example:  1
Number of rules generated:  5
Number of rules followed:  4
1st rule followed  1
2nd rule followed  0
3rd rule followed  1
Example exists in data:  False


## Tree of Thought
Tree of thought will open a given amount of branches and for each one of them an explanation will be generated like in the previous examples

In [3]:
exp_m = ToTLLMExplanation4CFs(model = model, #Load the model we want to explain
                        model_description = """ML-system that predicts wether a person will earn more than 50k $ a year""", # brief explanation of the ML model
                        backend='sklearn', # Framework used to build the model (used to generate counterfactuals)
                        dataset_info=string_info(train_dataset.columns, helpers.get_adult_data_info()) , # string information about the dataset
                        continuous_features=['age', 'hours_per_week'], # Necessary for the counterfactual generation
                        outcome_name= 'income', #Necessary for counterfactual generation
                        training_set=train_dataset, #Necessary for counterfactual generation
                        test_set= test_dataset, #Necessary to  check novelty of the evaluation example
                        llm='gpt-4o', #LLM used, works with Langchain
                        prompt_type='zero', # zero or one
                        n_counterfactuals=5, #Number of counterfactuals used in the explanation 
                        user_input=False, #Human in the loop helping select the causes
                        branches = 3
                    )
exp_m.fit()

out, explanation, code2, final_cf, code3, final_df, prediction, n_rules,rules_followed, first_rule, second_rule,third_rule, in_data = exp_m.explain_evaluate(user_data = test_df.iloc[[2]], verbose = False,return_all=True) 

100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


After this process is done, we join all the examples in order ro feed them to the LLM

In [4]:
print(out)



System1:
Rules:
To extract the most important rules that differentiate between the negative and positive outcomes from the provided data, we need to identify the key changes between the negative assessment and the positive counterfactuals. Let's analyze the differences:

### Negative Assessment Outcome:
- **Age**: 41
- **Workclass**: Private
- **Education**: School
- **Marital Status**: Married
- **Occupation**: Blue-Collar
- **Race**: White
- **Gender**: Male
- **Hours per Week**: 30
- **Income**: 0

### Positive Counterfactual Outcomes:
1. **Case 1**:
   - **Age**: 61
   - **Workclass**: Private
   - **Education**: Bachelors
   - **Marital Status**: Married
   - **Occupation**: Blue-Collar
   - **Race**: White
   - **Gender**: Male
   - **Hours per Week**: 30
   - **Income**: 1

2. **Case 2**:
   - **Age**: 41
   - **Workclass**: Private
   - **Education**: Masters
   - **Marital Status**: Married
   - **Occupation**: Blue-Collar
   - **Race**: Other
   - **Gender**: Male
   - **Ho

Using all this information, the LLM generates an explanation.

In [5]:
print(explanation)

Sure, here's a clear and straightforward explanation to help you understand what steps you can take to improve your chances of earning more than $50,000 a year.

---

### Key Steps to Improve Your Income

Based on the analysis of your current situation and the comparison with others who earn more than $50,000 a year, here are some important steps you can take:

#### 1. **Pursue Higher Education**
   - **Why It's Important**: The most consistent and crucial factor in achieving a higher income is furthering your education. All the positive cases had individuals with higher education levels such as a Bachelor's, Master's, or Professional degree.
   - **Action to Take**: Consider investing in further education. Enroll in and complete a Bachelor's, Master's, or Professional school program. This step alone can significantly increase your earning potential.

#### 2. **Gain More Work Experience Over Time**
   - **Why It's Important**: In some cases, individuals who were older and had more work

Again, we follow the same process for the evaluation

In [6]:
print(code2)

import pandas as pd

# Creating the DataFrame with an example that would be classified as the positive class
df = pd.DataFrame({
    'age': [45],
    'workclass': ['Private'],
    'education': ['Bachelors'],
    'marital_status': ['Married'],
    'occupation': ['Professional'],
    'race': ['White'],
    'gender': ['Male'],
    'hours_per_week': [40],
    'income': [1]  # This indicates the positive class (>50K)
})

# Save the DataFrame to a CSV file
df.to_csv('temp_csv.csv', index=False)


In [7]:
final_cf

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
0,45,Private,Bachelors,Married,Professional,White,Male,40


In [8]:
print(code3)

import pandas as pd

# Load the example from temp_csv
df = pd.read_csv('temp_csv.csv')

# Define the rules
rules = [
    {
        'Rule': 'Higher Education Level',
        'Importance': 5,
        'Check': lambda x: x['education'] in ['Bachelors', 'Masters', 'Prof-school', 'Doctorate']
    },
    {
        'Rule': 'Older Age',
        'Importance': 2,
        'Check': lambda x: x['age'] >= 52
    },
    {
        'Rule': 'Marital Status Changed to Separated',
        'Importance': 1,
        'Check': lambda x: x['marital_status'] == 'Separated'
    },
    {
        'Rule': 'Occupation Shift to Service',
        'Importance': 1,
        'Check': lambda x: x['occupation'] == 'Service'
    },
    {
        'Rule': 'Race Other than White',
        'Importance': 1,
        'Check': lambda x: x['race'] == 'Other'
    },
    {
        'Rule': 'Professional and White-Collar occupations lead to higher income',
        'Importance': 2,
        'Check': lambda x: x['occupation'] in ['Professiona

In [9]:
final_df

,Rule,Importance,In explanation
0,Higher Education Level,5,1
1,Older Age,2,0
2,Marital Status Changed to Separated,1,0
3,Occupation Shift to Service,1,0
4,Race Other than White,1,0
5,Professional and White-Collar occupations lead...,2,1
6,Marital status might have a minor influence bu...,5,0
7,Hours Per Week > 30,2,1
8,Combination Rule (Bachelors or higher AND Hour...,2,1


In [ ]:
print('Prediction of the generated example: ', prediction)
print('Number of rules generated: ', n_rules)
print('Number of rules followed: ', rules_followed)
print('1st rule followed ', first_rule)
print('2nd rule followed ', second_rule)
print('3rd rule followed ', third_rule)
print('Example exists in data: ', in_data)

Prediction of the generated example:  1
Number of rules generated:  14
Number of rules followed:  4
1st rule followed  0
2nd rule followed  0
3rd rule followed  0
Example exists in data:  False


As we can see, the rule set is much bigger now, so different causes can be selected

In [10]:
test_df.iloc[[2]]

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week
2,41,Private,School,Married,Blue-Collar,White,Male,30
